In [2]:
!pip install gradio pymupdf sentence-transformers transformers qdrant-client accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [3]:
from getpass import getpass
from huggingface_hub import login
access_token = getpass("Enter your Hugging Face access token:")
login(access_token)

Enter your Hugging Face access token:··········


In [5]:
import os
import torch
import gradio as gr
import shutil
import fitz
import qdrant_client
from qdrant_client.models import PointStruct, VectorParams, Distance
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
from uuid import uuid4

# ------------------ Model Setup ------------------

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Embedding Model (E5 small v2)
embedding_model = SentenceTransformer("intfloat/e5-small-v2", device=DEVICE)

# Language Model (Google Gemma 2B IT)
lm_tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
lm_model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", torch_dtype=torch.float16)
lm_model.to(DEVICE)

# ------------------ Vector DB Setup ------------------

qdrant = qdrant_client.QdrantClient(":memory:")
qdrant_collection = "docs"

qdrant.recreate_collection(
    collection_name=qdrant_collection,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

# ------------------ File Processing ------------------

def extract_chunks(file_path):
    filename = os.path.basename(file_path)
    chunks = []

    if filename.endswith(".pdf"):
        doc = fitz.open(file_path)
        for page_num in range(len(doc)):
            text = doc[page_num].get_text()
            if not text:
                continue
            sentences = text.split(". ")
            for i in range(0, len(sentences), 5):
                chunk = ". ".join(sentences[i:i+5])
                if chunk.strip():
                    chunks.append({
                        "text": chunk.strip(),
                        "metadata": {
                            "filename": filename,
                            "page": page_num + 1,
                            "chunk_id": f"{filename}-p{page_num+1}-c{i//5 + 1}"
                        }
                    })

    elif filename.endswith(".txt"):
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()
        sentences = text.split(". ")
        for i in range(0, len(sentences), 5):
            chunk = ". ".join(sentences[i:i+5])
            if chunk.strip():
                chunks.append({
                    "text": chunk.strip(),
                    "metadata": {
                        "filename": filename,
                        "page": 1,
                        "chunk_id": f"{filename}-p1-c{i//5 + 1}"
                    }
                })
    return chunks

# ------------------ Indexing ------------------

def index_file(file):
    filename = os.path.basename(file.name)
    shutil.copy(file.name, filename)
    chunks = extract_chunks(filename)

    points = []
    for chunk in chunks:
        embedding = embedding_model.encode(chunk["text"]).tolist()
        points.append(PointStruct(
            id=str(uuid4()),
            vector=embedding,
            payload={
                "text": chunk["text"],
                "filename": chunk["metadata"]["filename"],
                "page": chunk["metadata"]["page"],
                "chunk_id": chunk["metadata"]["chunk_id"]
            }
        ))

    qdrant.upsert(collection_name=qdrant_collection, points=points)
    os.remove(filename)
    return f"✅ Indexed {len(points)} chunks from {filename}"

# ------------------ RAG Answering ------------------

def answer_query(query):
    query_embedding = embedding_model.encode(query).tolist()
    hits = qdrant.search(
        collection_name=qdrant_collection,
        query_vector=query_embedding,
        limit=3
    )

    context = ""
    sources = []
    for hit in hits:
        payload = hit.payload
        context += payload["text"] + "\n"
        sources.append(f'📄 {payload["filename"]} | Page {payload["page"]} | Chunk: {payload["chunk_id"]}')

    prompt = f"""Answer the user's question based on the information provided below:\n\nContext:\n{context}\n\nQuestion: {query}\nAnswer:"""

    inputs = lm_tokenizer(prompt, return_tensors="pt").to(DEVICE)
    outputs = lm_model.generate(**inputs, max_new_tokens=200, do_sample=True, temperature=0.7)
    answer = lm_tokenizer.decode(outputs[0], skip_special_tokens=True).split("Answer:")[-1].strip()

    return answer, "\n".join(sources)

# ------------------ Gradio UI ------------------

with gr.Blocks() as demo:
    gr.Markdown("## 🤖 Real-time RAG Chatbot (Gemma + E5 + Qdrant)")

    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(height=400)
            msg = gr.Textbox(placeholder="Ask your question...", label="User Prompt")
            send_btn = gr.Button("Submit", variant="primary")
        with gr.Column(scale=1):
            file_upload = gr.File(label="Upload PDFs or TXT", file_types=[".pdf", ".txt"])
            upload_btn = gr.Button("📁 Upload & Index")
            status = gr.Textbox(label="Status")

    def handle_query(message, chat_history):
        answer, sources = answer_query(message)
        chat_history.append((message, f"{answer}\n\n📌 Sources:\n{sources}"))
        return "", chat_history

    def handle_upload(file):
        return index_file(file)

    send_btn.click(handle_query, [msg, chatbot], [msg, chatbot])
    upload_btn.click(handle_upload, file_upload, status)

demo.launch()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/tmp/ipython-input-5-2290522284.py:29: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(
/tmp/ipython-input-5-2290522284.py:133: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=400)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bf938274b637f2e63f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
